# Training the autoencoder

This file contains sample code for training the autoencoder used for dimensionali reduction in our stage 1 clustering (whole data)

Load embedding data -- huge!

In [1]:
from utils.embeddings import get_sbert_embeddings
from pathlib import Path
DATA_DIR = Path('/data/blockchain-interoperability/blockchain-social-media/twitter-data')

embeddings = get_sbert_embeddings(
    snapshot_path = DATA_DIR/'snapshots',
    embeddings_path = DATA_DIR/'embeddings',
)

loading from cache


Turn into torch dataloader

This handles a bunch of things for you so you don't need to manually write code

In [2]:
from torch.utils.data import DataLoader

# torch utility for shuffling/batching training data
loader = DataLoader(embeddings,batch_size=1024, shuffle=True)

Running the training loop.

We want the loss to start settling at some value. Find the right epoch to train til.

In [8]:
from utils.autoencoders import LinearAutoEncoder
from torch.optim import SGD

autoenc = LinearAutoEncoder().cuda()
# you can try different values for learing rate and weight_decay, and also try out Adam instaed of SGD

# learning_rate: higher means more likely to overfit, faster weight updates
# weight_decay: higher (closer to 1) means less weight decay. The same weights are preserved more. 
optimizer = SGD(autoenc.parameters(),lr=5e-2,momentum=0.9,weight_decay=1e-4)


In [9]:
# train autoencoder
from utils.autoencoders import RMSELoss
import torch.nn.functional as F
from tqdm.notebook import tqdm

# we could use dataloader, but all the data is loaded already so no point..
# batch_size = 1000


for epoch in range(10):
    autoenc.train()
    epoch_loss = 0
    # iterate thru the dataloader
    for batch in tqdm(loader):
        autoenc.zero_grad()
        batch = batch.cuda()
        out = autoenc(batch)

        loss = F.mse_loss(batch,out)
        # loss = RMSELoss(batch,out)
        epoch_loss += loss.item()
        loss.backward()
        optimizer.step()
    print(f'epoch_loss: {epoch_loss}')
        

        # break
    # pass
    # break

  0%|          | 0/14623 [00:00<?, ?it/s]

epoch_loss: 73.21462510945275


  0%|          | 0/14623 [00:00<?, ?it/s]

epoch_loss: 26.679077763459645


  0%|          | 0/14623 [00:00<?, ?it/s]

...

epoch_loss: 26.617966204532422


  0%|          | 0/14623 [00:00<?, ?it/s]

epoch_loss: 26.61746647756081


  0%|          | 0/14623 [00:00<?, ?it/s]

epoch_loss: 26.617185704992153


In [10]:
import torch
# save the best model! ensure you write it under the data directory, so we don't include the huge file in git

torch.save(autoenc.state_dict(), DATA_DIR/'autoenc_10_epoch.pkl')

In [ ]:
something = [0,1,2,3,4]

